# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import hvplot.pandas
import pandas as pd
import requests
import json
import os
from pprint import pprint

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [93]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/CitiesByYK.csv")
                           
# Display sample data and remove na`s
###creating own criteria to plan for the best vacation with mild wind
city_data_clean = city_data_df.dropna()
city_data_clean.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ochsenhausen,48.0667,9.9500,22.60,67,40,1.03,DE,1696265987
1,1,port-aux-francais,-49.3500,70.2167,0.99,61,100,14.93,TF,1696265988
2,2,puerto natales,-51.7236,-72.4875,9.25,61,75,5.66,CL,1696265989
3,3,bethel,41.3712,-73.4140,25.08,64,0,2.06,US,1696265903
4,4,talnakh,69.4865,88.3972,-2.66,85,100,7.80,RU,1696265991
5,5,vila velha,-20.3297,-40.2925,28.97,83,20,4.63,BR,1696265992
6,6,tralee,52.2704,-9.7026,15.14,77,40,2.57,IE,1696265994
7,7,fort mcmurray,56.7268,-111.3810,9.73,87,100,1.03,CA,1696265709
8,8,pangoa,-12.1167,-73.0000,33.06,42,18,2.66,PE,1696265995
9,9,luanda,-8.8368,13.2343,25.00,78,40,6.17,AO,1696265735


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [94]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_clean.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.95,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [95]:
###creating own criteria to plan for the best vacation with mild wind, using extra columns in this line

# Narrow down cities that fit criteria and drop any results with null values
#vacation, needs a bit of wind
narrow_df = pd.DataFrame(city_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness", "Country", "Lat", "Lng", "Humidity"])
best_temp = (narrow_df["Max Temp"] <= 35) & (narrow_df["Max Temp"] > 20)
best_wind_to_sail = (narrow_df["Wind Speed"] > 2) & (narrow_df["Wind Speed"] < 5)
best_cloud = narrow_df["Cloudiness"] == 0
country = narrow_df["Country"]
lat = narrow_df["Lat"]
lng = narrow_df["Lng"]
humidity = narrow_df["Humidity"]

# Drop any rows with null values
narrow_df = narrow_df.dropna()

# Display sample data with best mild wind  
ideal_narrow = narrow_df[best_temp & best_wind_to_sail & best_cloud]
ideal_narrow

,City,Max Temp,Wind Speed,Cloudiness,Country,Lat,Lng,Humidity
3,bethel,25.08,2.06,0,US,41.3712,-73.4140,64
89,kahuta,23.80,4.01,0,PK,33.5913,73.3868,28
135,mahdia,24.97,4.46,0,TN,35.5047,11.0622,64
149,rio grande,22.06,3.21,0,BR,-32.0350,-52.0986,67
159,taunsa,29.05,2.39,0,PK,30.7048,70.6505,13
212,formosa,31.09,4.12,0,AR,-26.1775,-58.1781,38
234,heroica caborca,23.53,3.62,0,MX,30.7167,-112.1500,47
250,colonia,25.46,2.57,0,DE,50.9333,6.9500,56
254,silver city,23.98,4.02,0,US,32.7701,-108.2803,50
295,loralai,20.27,3.26,0,PK,30.3705,68.5980,9


### Step 3: Create a new DataFrame called `hotel_df`.

In [96]:
###using new Ideal Narrow DF to plan the best vacation with mild wind 

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_narrow.copy()
hotel_df = pd.DataFrame(ideal_narrow, columns = ["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,64,
89,kahuta,PK,33.5913,73.3868,28,
135,mahdia,TN,35.5047,11.0622,64,
149,rio grande,BR,-32.0350,-52.0986,67,
159,taunsa,PK,30.7048,70.6505,13,
212,formosa,AR,-26.1775,-58.1781,38,
234,heroica caborca,MX,30.7167,-112.1500,47,
250,colonia,DE,50.9333,6.9500,56,
254,silver city,US,32.7701,-108.2803,50,
295,loralai,PK,30.3705,68.5980,9,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [111]:
radius = 10000
params = {
    "types":"hotel",
    "keyword": hotels,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotels in this area"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
kahuta - nearest hotel: No hotels in this area
mahdia - nearest hotel: La Maison Bleue
rio grande - nearest hotel: Hotel Vila Moura Executivo
taunsa - nearest hotel: No hotels in this area
formosa - nearest hotel: Hotel Hostal del Rey
heroica caborca - nearest hotel: Hotel Posada del Desierto
colonia - nearest hotel: Wasserturm Hotel Cologne
silver city - nearest hotel: Murray Hotel
loralai - nearest hotel: No hotels in this area
lata - nearest hotel: Residencial Botânico
dalbandin - nearest hotel: SF Al-dawood Restaurant
tazacorte - nearest hotel: App Leyma
hun - nearest hotel: فندق الهروج
meiderich - nearest hotel: Hotel am Freihafen
north madison - nearest hotel: Clifty Inn
retreat - nearest hotel: No hotels in this area


,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,64,Hampton Inn Danbury
89,kahuta,PK,33.5913,73.3868,28,No hotels in this area
135,mahdia,TN,35.5047,11.0622,64,La Maison Bleue
149,rio grande,BR,-32.0350,-52.0986,67,Hotel Vila Moura Executivo
159,taunsa,PK,30.7048,70.6505,13,No hotels in this area
212,formosa,AR,-26.1775,-58.1781,38,Hotel Hostal del Rey
234,heroica caborca,MX,30.7167,-112.1500,47,Hotel Posada del Desierto
250,colonia,DE,50.9333,6.9500,56,Wasserturm Hotel Cologne
254,silver city,US,32.7701,-108.2803,50,Murray Hotel
295,loralai,PK,30.3705,68.5980,9,No hotels in this area


In [127]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    scale = 2,
    hover_cols = ["Country", "Hotel Name"]
)
# Save the figure
plt.savefig("../output_data/ideal_hotel_map.png")

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)

<Figure size 640x480 with 0 Axes>

In [ ]:
# #target_url = "https://api.geoapify.com/v2/places"
# #response = requests.get(target_url, params=params)
# #print(response.url)

# #places_data = response.json()
# #print(json.dumps(places_data, indent=4, sort_keys=True))